#### Datos de partida:
¿Cuántas clases hay? ¿Cuántos ejemplos de cada? ¿Cuál es el acierto mínimo esperado?
¿Cómo se normalizan las imágenes de entrada? ¿se te ocurre una opción mejor? Si es así, ¿cuál? Prúebala
¿Cuál es el formato de las imágenes de entrada (dimensiones)? ¿Por qué es necesario hacer la operación flatten en dichas imágenes para aplicar la regresión logística?

Hay dos clases en nuestro grupo de datos, muestras malignas y muestras benignas, que se representan el las imagenes de la celda 4.
Tenemos 6691 muestras de las cuales 4165 (62.25%) benignas y 2526 (37,75%) malignas. El acierto mínomo de esperado será que clasifique todas las muestras en una sola clase, 